In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, regexp_replace, lit, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

custom_schema = (StructType([
    StructField("title", StringType(), False),
    StructField("subtitle", StringType(), True),
    StructField("description", StringType(), True),
    StructField("discount", DoubleType(), True),
    StructField("old_price", DoubleType(), True),
    StructField("new_price", DoubleType(), False),
    StructField("date", TimestampType(), True),  # Espace supprimé avant "date"
    StructField("images", StringType(), True),
    StructField("category", StringType(), True)
])
)

In [0]:

# Function to format the lines in csv file
def line_formating():
    col_to_modify = ["description", "images"]
    df = spark.read.format("csv").schema(custom_schema).option("header", "true").option("inferSchema", "true").option("delimiter", ";").option("encoding", "UTF-8").load("/Volumes/workspace/default/yellow_taxi/*.csv")
    for c in df.columns:
        if c in col_to_modify:
            if c == "images":
                df = df.withColumn(c, expr(f"replace({c}, ';', ',')"))
            df = df.withColumn(c, expr(f"replace({c}, '\"\"', '\"')"))
            df = df.withColumn(c, regexp_replace(c, '^"|"$', ''))
    return df

In [0]:
dbutils.fs.rm("/Volumes/workspace/default/lidl_products/lidl_product.parquet", True)

In [0]:
# save df to parquet file
line_formating().write.format("parquet").mode("overwrite").save("/Volumes/workspace/default/lidl_products/lidl_product.parquet")
